#Pathweigh Analysis.


1. Run all the following cells.
2. Input file is should be named input.csv and can be either microarray or next gen RNA sequence data.
3. Configure the following parameter: is_rnaseq.
3. Configure sample_num.
4. Configure path_id. Use the following table in order to configure a path ID for analysis and export:
https://github.com/zurkin1/Pathweigh/blob/master/README.md.

You can use your own file on Colab by uploading it to the server.

In [ ]:
#Default configuration.
is_rnaseq = False
sample_num = 0
path_id = 708708 #1-and 2-Methylnaphthalene degradation(Kegg).

!rm -r Pathweigh
!git clone https://github.com/zurkin1/Pathweigh.git
!mv Pathweigh/code/*.* .

Cloning into 'Pathweigh'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 217 (delta 21), reused 37 (delta 10), pack-reused 163
Receiving objects: 100% (217/217), 13.90 MiB | 7.49 MiB/s, done.
Resolving deltas: 100% (105/105), done.


## Calculate UDP per Gene and Paths Activity.

In [ ]:
from udp import *

udp = calc_udp_multi_process(is_rnaseq = is_rnaseq)

from activity import *

activity_obj = path_activity(udp, is_rnaseq = is_rnaseq)
activity = activity_obj.calc_activity_consistency_multi_process() #Output is saved to output_activity.csv.

activity_obj.activity.head()

Wed Jan 13 14:06:16 2021 Calculate UDP, is_rnaseq: False
............Wed Jan 13 14:06:48 2021 Done.
Wed Jan 13 14:06:50 2021 Init activity object
Wed Jan 13 14:06:50 2021 Calculate activity and consistency...
...Wed Jan 13 14:06:57 2021 Done.


,sampleID,path_name,pathID,Activity,Consistency,molRole
0,GSM718769,west nile virus(BioCarta),100001,0.701975,0.705708,input
1,GSM718769,west nile virus(BioCarta),100001,0.701975,0.705708,output
2,GSM718769,west nile virus(BioCarta),100001,0.701975,0.705708,agent
3,GSM718769,wnt signaling pathway(BioCarta),100002,0.185143,0.349168,input
4,GSM718769,wnt signaling pathway(BioCarta),100002,0.185143,0.349168,output


##Draw a KGML Map For Path Activity and Download a KGML File.

In [ ]:
activity_obj.graphparser(path_id=path_id, sample_num=sample_num)

xml_result = activity_obj.xmlparser(path_id=path_id, sample_num=sample_num)

Create Kegg XML for path: 708708, sample: 0


In [ ]:
import bs4

print(bs4.BeautifulSoup(xml_result).prettify())

<?xml version="1.0"?>
<!DOCTYPE pathway SYSTEM "https://www.kegg.jp/kegg/xml/KGML_v0.7.2_.dtd">
<html>
 <body>
  <pathway name="path:hsa708708" number="111" org="hsa" title="1- and 2-Methylnaphthalene degradation(Kegg)">
   <entry id="708648" link="https://www.kegg.jp/dbget-bin/www_bget?hsa:4967+hsa:708648" name="hsa:4967 hsa:fm_708637" type="protein">
    <graphics bgcolor="#42ECEF" fgcolor="#FF0000" height="17" name="fm_708637" type="rectangle" width="60" x="1.0">
    </graphics>
   </entry>
   <entry id="708709" link="https://www.kegg.jp/dbget-bin/www_bget?C00158" name="708709" type="reaction">
    <graphics bgcolor="#FFF8F8" fgcolor="#FF0000" height="15" name="reaction" type="circle" width="60" x="1.0" y="1.0">
    </graphics>
   </entry>
   <relation entry1="708648" entry2="708709" type="PPrel">
    <subtype name="activation" value="--&gt;">
    </subtype>
   </relation>
   <entry id="708651" link="https://www.kegg.jp/dbget-bin/www_bget?hsa:4967+hsa:708651" name="hsa:4967 hsa:2-Na